Notebook com três tipos de data augumentation: espelhamento horizontal, zoom e rotação de 90 graus.

Caminhos de entrada e saída

In [25]:
input_base_dir = r'C:\Mestrado\Materias\2024-02-Sistemas para Processamento Multimidia\seminario\image-segmentation\dataset\nematode'
output_base_dir = r'C:\Mestrado\Materias\2024-02-Sistemas para Processamento Multimidia\seminario\image-segmentation\dataset\augumented-nematode'

Bibliotecas necessárias

In [27]:
import os
import json
import cv2
import numpy as np
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
from tqdm import tqdm

Criação das pastas de saída para train, valid, test

In [17]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

Função para aplicar zoom

In [19]:
def zoom_image(image, zoom_factor):
    height, width = image.shape[:2]
    new_width, new_height = int(width * zoom_factor), int(height * zoom_factor)
    image_zoomed = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_LINEAR)
    
    if zoom_factor > 1:
        # Recortar a imagem centralizada para o tamanho original
        start_x = (new_width - width) // 2
        start_y = (new_height - height) // 2
        return image_zoomed[start_y:start_y+height, start_x:start_x+width]
    else:
        # Preencher com bordas se o zoom for menor que 1
        border_x = (width - new_width) // 2
        border_y = (height - new_height) // 2
        return cv2.copyMakeBorder(image_zoomed, border_y, border_y, border_x, border_x, cv2.BORDER_CONSTANT, value=[0, 0, 0])

Função para aplicar rotação e ajustar os labels

In [20]:
def rotate_image(image, angle):
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, matrix, (w, h))
    return rotated

Função para aplicar espelhamento e ajustar os labels

In [21]:
def mirror_image(image):
    return cv2.flip(image, 1)

Função para atualizar as anotações JSON de acordo com a transformação

In [23]:
def update_annotations(json_file, transformation, img_shape):
    with open(json_file, 'r') as f:
        data = json.load(f)

    height, width = img_shape[:2]
    for shape in data['shapes']:
        points = shape['points']
        
        if transformation == 'mirror':
            for point in points:
                point[0] = width - point[0]  # Espelha em relação ao eixo X
        
        elif transformation == 'rotate':
            center_x, center_y = width // 2, height // 2
            for point in points:
                # Rotaciona os pontos em torno do centro
                x_shifted = point[0] - center_x
                y_shifted = point[1] - center_y
                new_x = center_x - y_shifted  # Rotaciona 90 graus
                new_y = center_y + x_shifted
                point[0], point[1] = new_x, new_y

        elif transformation == 'zoom':
            zoom_factor = 1.2  # Exemplo de zoom 20%
            for point in points:
                point[0] = int(point[0] * zoom_factor)
                point[1] = int(point[1] * zoom_factor)

    return data


Função principal para processar as imagens e aplicar data augmentation

In [28]:
def augment_data(input_base_dir, output_base_dir):
    # Criar diretório de saída, se necessário
    create_dir(output_base_dir)
    
    subfolders = ['Train', 'Val', 'Test']  # Pastas que devem ser varridas

    for subfolder in subfolders:
        input_dir = os.path.join(input_base_dir, subfolder)
        output_dir = os.path.join(output_base_dir, subfolder)
        create_dir(output_dir)

        for root, _, files in os.walk(input_dir):
            for file in files:
                if file.endswith('.png') or file.endswith('.jpg'):
                    img_path = os.path.join(root, file)
                    img = cv2.imread(img_path)

                    # Caminhos para salvar as imagens transformadas
                    base_name, ext = os.path.splitext(file)
                    img_shape = img.shape

                    # Aplicar espelhamento
                    mirrored_img = mirror_image(img)
                    mirrored_json = update_annotations(os.path.join(root, base_name + '.json'), 'mirror', img_shape)
                    cv2.imwrite(os.path.join(output_dir, base_name + '_mirror' + ext), mirrored_img)
                    with open(os.path.join(output_dir, base_name + '_mirror.json'), 'w') as f:
                        json.dump(mirrored_json, f, indent=4)

                    # Aplicar rotação
                    rotated_img = rotate_image(img, 90)
                    rotated_json = update_annotations(os.path.join(root, base_name + '.json'), 'rotate', img_shape)
                    cv2.imwrite(os.path.join(output_dir, base_name + '_rotate' + ext), rotated_img)
                    with open(os.path.join(output_dir, base_name + '_rotate.json'), 'w') as f:
                        json.dump(rotated_json, f, indent=4)

                    # Aplicar zoom
                    zoomed_img = zoom_image(img, 1.2)
                    zoomed_json = update_annotations(os.path.join(root, base_name + '.json'), 'zoom', zoomed_img.shape)
                    cv2.imwrite(os.path.join(output_dir, base_name + '_zoom' + ext), zoomed_img)
                    with open(os.path.join(output_dir, base_name + '_zoom.json'), 'w') as f:
                        json.dump(zoomed_json, f, indent=4)

augment_data(input_base_dir, output_base_dir)